In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import jax
import jax.numpy as jnp
import optax
import flax
from flax.training import train_state, checkpoints
from flax_rbf.flax_rbf import *
from irbfn_mpc.model import WCRBFNet
from irbfn_mpc.irbfn_planner import IRBFNPlanner

2024-08-22 12:45:08.255057: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
import gymnasium as gym
from irbfn_mpc.planner_utils import intersect_point, nearest_point
import numpy as np
import yaml
import argparse

In [4]:
from IPython.display import HTML, display
%matplotlib inline

In [5]:
def rettrue(in_int):
    return True

env = gym.make(
    "f1tenth_gym:f1tenth-v0",
    config={
        "map": "Spielberg_blank",
        "observation_config": {"type": "dynamic_state"},
        "num_agents": 1,
        "control_input": "accl",
    },
    render_mode="rgb_array",
)
env = gym.wrappers.RecordVideo(env, "video_irbfn_nmpc_6regions", episode_trigger=rettrue)

/usr/local/lib/python3.12/dist-packages/f1tenth_gym/envs/action.py:155: UserWarning: Only one control mode specified, using accl for longitudinal control and defaulting to steering speed for steering
  warnings.warn(


In [6]:
track = env.unwrapped.track
waypoints = np.stack([track.centerline.xs, track.centerline.ys, track.centerline.vxs, track.centerline.yaws]).T

In [7]:
config_f = "configs/dnmpc_6regions_newdata_oldintloss.yaml"
ckpt = "ckpts/dnmpc_6regions_newdata_oldintloss/checkpoint_300"

In [8]:
planner = IRBFNPlanner(config_f, ckpt, track)

In [9]:
env.unwrapped.add_render_callback(planner.render_waypoints)
env.unwrapped.add_render_callback(planner.render_goal_state)
env.unwrapped.add_render_callback(planner.render_planner_sol)

In [13]:
poses = np.array(
    [
        [
            env.unwrapped.track.centerline.xs[0] + 1.0,
            env.unwrapped.track.centerline.ys[0] + 1.0,
            env.unwrapped.track.centerline.yaws[0],
        ]
    ]
)
obs, info = env.reset(options={"poses": poses})
done = False

step = 0

while not done and step < 10000:
    current_state = obs["agent_0"]
    accl, steerv = planner.plan(current_state)
    action = env.action_space.sample()
    # speed = current_state["linear_vel_x"] + pred_u[0, 0] * 0.1
    # steer = current_state["delta"] + pred_u[0, 5] * 0.1
    action[0] = [steerv, accl]

    obs, step_reward, done, truncated, info = env.step(action*0.9)
    step += 1
env.close()

Moviepy - Building video /home/irbfn/scripts/video_irbfn_nmpc_6regions/rl-video-episode-1.mp4.
Moviepy - Writing video /home/irbfn/scripts/video_irbfn_nmpc_6regions/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/irbfn/scripts/video_irbfn_nmpc_6regions/rl-video-episode-1.mp4


In [14]:
import glob
import io
import base64

for video_file in glob.glob("video_irbfn_nmpc_6regions/*.mp4"):
    video = io.open(video_file, "rb").read()
    encoded = base64.b64encode(video).decode("ascii")
    display(
        HTML(
            f"""<video width="800" height="auto" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>"""
        )
    )

In [ ]:
# testing on a grid
r_goal_x = np.linspace(0.1, 5.0, 10)
r_goal_y = np.linspace(-5.0, 5.0, 10)
